In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### **Crear un DataFrame a partir de un RDD**

In [ ]:
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2)) # Queremos que nos devuelva tuplas.

In [3]:
rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [4]:
df = rdd.toDF(['numero', 'cuadrado']) # Le damos una lista con los nombres de columnas que queremos en el DF.

In [5]:
df.printSchema()

root
 |-- numero: long (nullable = true)
 |-- cuadrado: long (nullable = true)



- long es el tipo de dato.

- nullable es si acepta nulos o no.

In [6]:
df.show()

+------+--------+
|numero|cuadrado|
+------+--------+
|     0|       0|
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
+------+--------+



- Crear un DF a partir de un RDD con schema.

In [7]:
rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

- 2 formas para crear el schema.

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

- Vía uno.

In [10]:
esquema1 = StructType(
    [
        StructField('id', IntegerType(), True), # Nombre de la columna, tipo de dato, si admite nulos o no.
        StructField('nombre', StringType(), True),
        StructField('saldo', DoubleType(), True)
    ]
)

Vía dos.

In [11]:
esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [12]:
df1 = spark.createDataFrame(rdd1, schema = esquema1)

In [13]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)



In [ ]:
df1.show() # Muestra por default las primeras 20 filas del DF.

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



In [15]:
df2 = spark.createDataFrame(rdd1, schema = esquema2)

In [16]:
df2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)



In [17]:
df2.show()

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



### **Crear un DF a partir de fuentes de datos**

- format() no es opcional, option() y schema() si.

- Crear un DF a partir de un **archivo** de **texto**.

In [18]:
df3 = spark.read.text('./data/data/dataTXT.txt')

In [21]:
df3.show()

+--------------------+
|               value|
+--------------------+
|Estamos en el cur...|
|En este capítulo ...|
|En esta sección e...|
|y en este ejemplo...|
+--------------------+



In [22]:
df3.show(truncate = False) # Para que muestre todo el texto.

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



- Crear un DF a partir de un **CSV**.

In [23]:
df4 = spark.read.csv('./data/data/dataCSV.csv')

In [24]:
df4.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|        _c0|          _c1|                 _c2|                 _c3|        _c4|                 _c5|                 _c6|    _c7|   _c8|     _c9|         _c10|                _c11|             _c12|            _c13|                _c14|                _c15|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|vid

- El nombre de las columnas lo ha generado pero en realidad el nombre de las columnas debería ser la primera fila.

In [25]:
df4 = spark.read.option('header', 'true').csv('./data/data/dataCSV.csv') # Le decimos que el título del campo es la primera fila.

In [26]:
df4.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

- Leer archivo de texto con un **delimitador diferente**.

In [28]:
df5 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/data/dataTab.txt')

In [29]:
df5.show()

+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



- El delimiter es que en este caso el archivo de texto no estaban los datos delimitados por comas sino por '|'

- Crear un DF a partir de un **JSON** proporcionando un **schema**.

In [30]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [31]:
json_schema = StructType(
    [
        StructField('color', StringType(), True),
        StructField('edad', IntegerType(), True),
        StructField('fecha', DateType(), True),
        StructField('pais', StringType(), True)
    ]
)

In [32]:
df6 = spark.read.schema(json_schema).json('./data/data/dataJSON.json')

In [33]:
df6.show()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+



In [34]:
df6.printSchema()

root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



- Crear un DF a partir de un archivo **parquet**.

In [ ]:
df7 = spark.read.parquet('./data/data/dataPARQUET.parquet')

In [36]:
df7.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

- Tenemos otra opción para la lectura.

In [ ]:
df8 = spark.read.format('parquet').load('./data/data/dataPARQUET.parquet')

In [39]:
df8.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



### **Trabajo con columnas**

- Al igual que en las operaciones con RDD, las operaciones estructuradas (con DF y SQL) tienen 2 categorías: **transformación** y **acción**.

- Los **DF** son **inmutables** y sus operaciones de transformación siempre devuelven un **DF nuevo**.

In [40]:
df9 = spark.read.parquet('./data/data/dataPARQUET.parquet')

In [41]:
df9.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



- **Primera** alternativa para **referirnos** a las columnas.

In [42]:
df9.select('title').show()

+--------------------+
|               title|
+--------------------+
|WE WANT TO TALK A...|
|The Trump Preside...|
|Racist Superman |...|
|Nickelback Lyrics...|
|I Dare You: GOING...|
|2 Weeks with iPho...|
|Roy Moore & Jeff ...|
|5 Ice Cream Gadge...|
|The Greatest Show...|
|Why the rise of t...|
|Dion Lewis' 103-Y...|
|(SPOILERS) 'Shiva...|
|Marshmello - Bloc...|
|Which Countries A...|
|SHOPPING FOR NEW ...|
|    The New SpotMini|
|One Change That W...|
|How does your bod...|
|HomeMade Electric...|
|Founding An Inbre...|
+--------------------+
only showing top 20 rows



- **Segunda** alternativa.

In [43]:
from pyspark.sql.functions import col

In [44]:
df9.select(col('title')).show()

+--------------------+
|               title|
+--------------------+
|WE WANT TO TALK A...|
|The Trump Preside...|
|Racist Superman |...|
|Nickelback Lyrics...|
|I Dare You: GOING...|
|2 Weeks with iPho...|
|Roy Moore & Jeff ...|
|5 Ice Cream Gadge...|
|The Greatest Show...|
|Why the rise of t...|
|Dion Lewis' 103-Y...|
|(SPOILERS) 'Shiva...|
|Marshmello - Bloc...|
|Which Countries A...|
|SHOPPING FOR NEW ...|
|    The New SpotMini|
|One Change That W...|
|How does your bod...|
|HomeMade Electric...|
|Founding An Inbre...|
+--------------------+
only showing top 20 rows



### **Transformaciones: funciones select() y selectExpr()**

- **select()**

In [45]:
df10 = spark.read.parquet('./data/data/dataPARQUET.parquet')

In [46]:
df10.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [47]:
from pyspark.sql.functions import col

In [48]:
df10.select(col('video_id')).show()

+-----------+
|   video_id|
+-----------+
|2kyS6SvSYSE|
|1ZAPwfrtAFY|
|5qpjK5DgCt4|
|puqaWrEC7tY|
|d380meD0W0M|
|gHZ1Qz0KiKM|
|39idVpFF7NQ|
|nc99ccSXST0|
|jr9QtXwC9vc|
|TUmyygCMMGA|
|9wRQljFNDW8|
|VifQlJit6A0|
|5E4ZBSInqUU|
|GgVmn66oK_A|
|TaTleo4cOs8|
|kgaO45SyaO4|
|ZAQs-ctOqXQ|
|YVfyYrEmzgM|
|eNSN6qet1kE|
|B5HORANmzHw|
+-----------+
only showing top 20 rows



In [ ]:
df10.select('video_id', 'trending_date').show() # La otra forma.

+-----------+-------------+
|   video_id|trending_date|
+-----------+-------------+
|2kyS6SvSYSE|     17.14.11|
|1ZAPwfrtAFY|     17.14.11|
|5qpjK5DgCt4|     17.14.11|
|puqaWrEC7tY|     17.14.11|
|d380meD0W0M|     17.14.11|
|gHZ1Qz0KiKM|     17.14.11|
|39idVpFF7NQ|     17.14.11|
|nc99ccSXST0|     17.14.11|
|jr9QtXwC9vc|     17.14.11|
|TUmyygCMMGA|     17.14.11|
|9wRQljFNDW8|     17.14.11|
|VifQlJit6A0|     17.14.11|
|5E4ZBSInqUU|     17.14.11|
|GgVmn66oK_A|     17.14.11|
|TaTleo4cOs8|     17.14.11|
|kgaO45SyaO4|     17.14.11|
|ZAQs-ctOqXQ|     17.14.11|
|YVfyYrEmzgM|     17.14.11|
|eNSN6qet1kE|     17.14.11|
|B5HORANmzHw|     17.14.11|
+-----------+-------------+
only showing top 20 rows



- select() presenta la desventaja de que no puedo construir expresiones dentro. Por ejemplo esto dará error:

In [ ]:
df10.select(
    'likes',
    'dislikes',
    ('likes' - 'dislikes') # Una nueva columna que sea la resta de likes y dislikes.
)

- Tenemos que hacerlo con 'col'

In [50]:
df10.select(
    col('likes'),
    col('dislikes'),
    (col('likes') - col('dislikes')).alias('aceptacion')
).show()

+------+--------+----------+
| likes|dislikes|aceptacion|
+------+--------+----------+
| 57527|    2966|   54561.0|
| 97185|    6146|   91039.0|
|146033|    5339|  140694.0|
| 10172|     666|    9506.0|
|132235|    1989|  130246.0|
|  9763|     511|    9252.0|
| 15993|    2445|   13548.0|
| 23663|     778|   22885.0|
|  3543|     119|    3424.0|
| 12654|    1363|   11291.0|
|   655|      25|     630.0|
|  1576|     303|    1273.0|
|114188|    1333|  112855.0|
|  7848|    1171|    6677.0|
|  7473|     246|    7227.0|
|  9419|      52|    9367.0|
|  8011|     638|    7373.0|
|  5398|      53|    5345.0|
| 11963|      36|   11927.0|
|  8421|     191|    8230.0|
+------+--------+----------+
only showing top 20 rows



- **selectExpr**

In [ ]:
df10.selectExpr('likes', 'dislikes', '(likes - dislikes) as aceptacion').show() # La otra forma.

+------+--------+----------+
| likes|dislikes|aceptacion|
+------+--------+----------+
| 57527|    2966|   54561.0|
| 97185|    6146|   91039.0|
|146033|    5339|  140694.0|
| 10172|     666|    9506.0|
|132235|    1989|  130246.0|
|  9763|     511|    9252.0|
| 15993|    2445|   13548.0|
| 23663|     778|   22885.0|
|  3543|     119|    3424.0|
| 12654|    1363|   11291.0|
|   655|      25|     630.0|
|  1576|     303|    1273.0|
|114188|    1333|  112855.0|
|  7848|    1171|    6677.0|
|  7473|     246|    7227.0|
|  9419|      52|    9367.0|
|  8011|     638|    7373.0|
|  5398|      53|    5345.0|
| 11963|      36|   11927.0|
|  8421|     191|    8230.0|
+------+--------+----------+
only showing top 20 rows



In [53]:
df10.selectExpr('count(distinct (video_id)) as videos').show()

+------+
|videos|
+------+
|  6837|
+------+



### **Transformaciones: funciones filter() y where()**

In [54]:
df11 = spark.read.parquet('./data/data/dataPARQUET.parquet')

- **filter()**

In [55]:
from pyspark.sql.functions import col

In [56]:
df11.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [57]:
df11.filter(col('video_id') == '2kyS6SvSYSE').show() # Nos trae las filas que coinciden con ese valor de video_id.

+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|           tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| CaseyNeistat|         22|2017-11-13T17:13:...|SHANtell martin| 748374|57527|    2966|        15954|https://i.ytimg.c...|            False|           False|                 False|SHANTELL'S CHANNE...|
|2ky

- **where()**

- Es muy similar a filter()

In [59]:
df12 = spark.read.parquet('./data/data/dataPARQUET.parquet').where(col('trending_date') != '17.14.11')

In [60]:
df12.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|            video_id|       trending_date|               title|       channel_title|         category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|\nCook with confi...|             recipes|              videos| and restaurant g...| dining destinations| and hosting idea...|            

In [62]:
df13 = spark.read.parquet('./data/data/dataPARQUET.parquet').where(col('likes') > 5000)

In [64]:
df13.filter((col('trending_date') != '17.14.11') & (col('likes') > 7000)).show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|YvfYK0EEhK4|     17.15.11|Brent Pella - Why...|         Brent Pella|         23|2017-11-14T15:32:...|"spirit airlines"...| 462490| 14132|     795|          666|https://i.ytimg.c...|            False|           Fal

- También podríamos haberlo hecho de esta otra forma.

In [65]:
df13.filter(col('trending_date') != '17.14.11').filter(col('likes') > 7000).show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|YvfYK0EEhK4|     17.15.11|Brent Pella - Why...|         Brent Pella|         23|2017-11-14T15:32:...|"spirit airlines"...| 462490| 14132|     795|          666|https://i.ytimg.c...|            False|           Fal